Import data

In [1]:
import pandas as pd
from keras import Sequential
from keras.layers import Dense, Activation, Dropout, LeakyReLU
from keras.optimizers import Adam
from keras import metrics
from keras.losses import MeanSquaredError
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data preprocessing

In [6]:
breast_dataset = pd.read_csv("./TheBostonHousingDataset.csv")
for column in breast_dataset.columns:
    print(column,len(pd.unique(breast_dataset[column].values)))
# 'CHAS', 'RAD' are categorical features

CRIM 504
ZN 26
INDUS 76
CHAS 2
NOX 81
RM 446
AGE 356
DIS 412
RAD 9
TAX 66
PTRATIO 46
B 357
LSTAT 455
MEDV 229


one-hot encoding

In [7]:
one_hot_CHAS = pd.get_dummies(breast_dataset['CHAS'],dtype=float)
one_hot_CHAS.rename(columns={0:'CHAS_0',1:'CHAS_1'},inplace=True)
one_hot_RAD= pd.get_dummies(breast_dataset['RAD'],dtype=float)
one_hot_CHAS

,CHAS_0,CHAS_1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
501,1.0,0.0
502,1.0,0.0
503,1.0,0.0
504,1.0,0.0


normalization

In [3]:
def normalization(X):
    X = (X - X.mean())/X.std()
    return X
breast_dataset.drop(columns=['CHAS','RAD'],inplace=True,axis=1)
breast_dataset.apply(normalization)


,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,LSTAT,MEDV
0,-0.419367,0.284548,-1.286636,-0.144075,0.413263,-0.119895,0.140075,-0.665949,-1.457558,0.440616,-1.074499,0.159528
1,-0.416927,-0.487240,-0.592794,-0.739530,0.194082,0.366803,0.556609,-0.986353,-0.302794,0.440616,-0.491953,-0.101424
2,-0.416929,-0.487240,-0.592794,-0.739530,1.281446,-0.265549,0.556609,-0.986353,-0.302794,0.396035,-1.207532,1.322937
3,-0.416338,-0.487240,-1.305586,-0.834458,1.015298,-0.809088,1.076671,-1.105022,0.112920,0.415751,-1.360171,1.181589
4,-0.412074,-0.487240,-1.305586,-0.834458,1.227362,-0.510674,1.076671,-1.105022,0.112920,0.440616,-1.025487,1.486032
...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.412820,-0.487240,0.115624,0.157968,0.438881,0.018654,-0.625178,-0.802418,1.175303,0.386834,-0.417734,-0.014440
502,-0.414839,-0.487240,0.115624,0.157968,-0.234316,0.288648,-0.715931,-0.802418,1.175303,0.440616,-0.500355,-0.210154
503,-0.413038,-0.487240,0.115624,0.157968,0.983986,0.796661,-0.772919,-0.802418,1.175303,0.440616,-0.982076,0.148655
504,-0.407361,-0.487240,0.115624,0.157968,0.724955,0.736268,-0.667776,-0.802418,1.175303,0.402826,-0.864446,-0.057932


Concat results

In [8]:
X = pd.concat([breast_dataset,one_hot_CHAS,one_hot_RAD],axis=1)
print(X.columns)

Index([   'CRIM',      'ZN',   'INDUS',    'CHAS',     'NOX',      'RM',
           'AGE',     'DIS',     'RAD',     'TAX', 'PTRATIO',       'B',
         'LSTAT',    'MEDV',  'CHAS_0',  'CHAS_1',         1,         2,
               3,         4,         5,         6,         7,         8,
              24],
      dtype='object')


In [15]:
y = X['MEDV']

X.drop(columns=['MEDV'],inplace=True)
# Split the data
train_val_x, test_x, train_val_y, test_y = train_test_split(X, y, test_size=0.1)
train_x, val_x, train_y, val_y = train_test_split(train_val_x, train_val_y, test_size=0.2)
train_data = (train_x.values, train_y.values)
val_data = (val_x.values, val_y.values)
test_data = (test_x.values, test_y.values)

In [25]:
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  CHAS_0   506 non-null    float64
 14  CHAS_1   506 non-null    float64
 15  1        506 non-null    float64
 16  2        506 non-null    float64
 17  3        506 non-null    float64
 18  4        506 non-null    float64
 19  5        506 non-null    float64
 20  6        506 non-null    float64
 21  7        506 non

design model

In [26]:
hidden_units = 15
input_size = 24
dropout=0.1
num_labels=1

model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size))
model.add(LeakyReLU())
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(LeakyReLU())
model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('relu'))
model.summary()
model.compile(loss=MeanSquaredError(),
                optimizer=Adam(learning_rate=0.0001),
                metrics=[metrics.mean_squared_error])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 13)                325       
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 13)                0         
                                                                 
 dropout_4 (Dropout)         (None, 13)                0         
                                                                 
 dense_7 (Dense)             (None, 13)                182       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 13)                0         
                                                                 
 dropout_5 (Dropout)         (None, 13)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                

In [27]:
def train(model, epoch, batch_size):
    train_x, train_y = train_data
    model.fit(train_x, train_y, epochs=epoch, batch_size=batch_size, verbose=1)

In [28]:
def validation(model):
    val_x, val_y = val_data
    val_loss = model.evaluate(val_x, val_y, verbose=0)
    print('\nValidation set: Average loss: {:.4f}'.format(val_loss[0]))

In [29]:
def test(model):
    test_x, test_y = test_data
    predictions = model.predict(test_x)
    mse = mean_squared_error(test_y, predictions)
    mae = mean_absolute_error(test_y, predictions)

    print('\nTest set: Average MSE: {:.4f}, Average MAE: {:.4f}\n'.format(mse, mae))

In [30]:
epoch_num = 500
batch_size = 32

def main(model):
    train(model, epoch=epoch_num, batch_size=batch_size)
    validation(model)
    test(model)
    model.save('model.h5')
main(model)

Epoch 1/500
12/12 [==============================] - 1s 3ms/step - loss: 1176.2904 - mean_squared_error: 1176.2904
Epoch 2/500
12/12 [==============================] - 0s 3ms/step - loss: 1065.4896 - mean_squared_error: 1065.4896
Epoch 3/500
12/12 [==============================] - 0s 3ms/step - loss: 845.7589 - mean_squared_error: 845.7589
Epoch 4/500
12/12 [==============================] - 0s 3ms/step - loss: 852.8884 - mean_squared_error: 852.8884
Epoch 5/500
12/12 [==============================] - 0s 3ms/step - loss: 856.9242 - mean_squared_error: 856.9242
Epoch 6/500
12/12 [==============================] - 0s 3ms/step - loss: 1074.4380 - mean_squared_error: 1074.4380
Epoch 7/500
12/12 [==============================] - 0s 2ms/step - loss: 806.3569 - mean_squared_error: 806.3569
Epoch 8/500
12/12 [==============================] - 0s 3ms/step - loss: 845.4172 - mean_squared_error: 845.4172
Epoch 9/500
12/12 [==============================] - 0s 3ms/step - loss: 841.2454 - mean_s